In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

In [16]:
import loader as load

In [2]:
SPARK_CONFIG

{'kerberos_auth': True,
 'numofcores': 9,
 'numofinstances': 9,
 'process_label': 'RECSYS_STORY',
 'replication_num': 1}

# Spark 

In [3]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

# Cheking oracle table 

In [17]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect()

In [18]:
db

In [19]:
table_name = 'recsys_story_asup_ratings'

In [20]:
ISKRA_TABLE = table_name

query = '''
select /*+ parallel (8) */
          *
FROM {}
where inn is not Null
and rownum <= 10000
'''.format(ISKRA_TABLE)
df = pd.read_sql(query, con=db.connection)

In [88]:
db.connection

<cx_Oracle.Connection to tech_iskra[iskra]@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=10.112.79.164)(PORT=1521))(CONNECT_DATA=(SID=iskra4)))>

In [21]:
df.head()

,INN,KORPKARTA,E_INV,RABOTA_RU,MERCHANT_ACQUIRING,E_ACQUIRING,KREDIT,CREDIT_CARD,BIP,ZARPLATA,LYURIST,SBERRATING,SPPU,CUSTOMWARRANTY,MYBUCH_ONLINE_SSO,SMS,TIME2PAY,SAMOINKASS,DEPOSIT,CASHORDER,LEASING,LOYALTY_BUSINESS,EVOTOR,INVOICE,BKI,...,MARKETS,PROPERTY_MANAGEMENT,DISTRESSED_ASSETS,STRUCTURALDEPOSITRANGEACCRUAL,OFD_PLATFORMA,SBER_BUSINESS_BOT,COOPERATEONLINE,E_INV_ARCHIVE,BESTBUSINESSP,PACKAGES,TAXFREE,CURRENCY_CONTROL_NOTIFICATIONS,INSALES,DCDGM,CUSTOMS_DUTIES,GOS_OBORON_ZAKAZ,DERIVATIVES,CREDIT_INSURANCE,BROKERAGE,SBER_TARGET,METRICS,DEPOSITDUALCURRENCY,CONTCHECK,CASH_INSURANCE,LOAD_DT
0,3662221896,0.731059,0.660728,0.681793,0.618091,0.614184,0.596907,0.593953,0.615717,0.583606,0.587329,0.563865,0.570781,0.533745,0.545147,0.534479,0.537019,0.528876,0.530532,0.529033,0.547879,0.526002,0.522353,0.523512,0.521486,...,0.502069,0.501334,0.502550,0.502155,0.501424,0.502664,0.501396,0.501272,0.501195,0.500204,0.500156,0.501182,0.503405,0.500050,0.500606,0.500117,0.500297,0.500172,0.500121,0.501552,None,None,0.500000,None,2021-10-05
1,3662251026,0.731059,0.660716,0.681762,0.618019,0.614095,0.596616,0.593721,0.615692,0.583490,0.587250,0.563625,0.570630,0.533663,0.544990,0.534279,0.536864,0.528661,0.530368,0.528898,0.547728,0.525816,0.522173,0.523345,0.521150,...,0.501856,0.501264,0.502337,0.501957,0.501264,0.502419,0.501207,0.501110,0.501006,0.500220,0.500139,0.500985,0.503144,0.500066,0.500496,0.500122,0.500247,0.500137,0.500120,0.501349,None,None,0.500000,None,2021-10-05
2,366310803686,0.731059,0.660737,0.681816,0.618154,0.614266,0.597111,0.594116,0.615744,0.583701,0.587387,0.564055,0.570923,0.533806,0.545282,0.534646,0.537137,0.529065,0.530673,0.529139,0.547989,0.526144,0.522504,0.523658,0.521746,...,0.502249,0.501377,0.502726,0.502322,0.501565,0.502875,0.501552,0.501410,0.501346,0.500162,0.500171,0.501355,0.503621,0.500038,0.500701,0.500097,0.500339,0.500191,0.500107,0.501721,None,None,0.500000,None,2021-10-05
3,366326312651,0.731059,0.660665,0.681617,0.617714,0.613722,0.595308,0.592683,0.615613,0.582986,0.586909,0.562553,0.569983,0.533301,0.544326,0.533408,0.536172,0.527738,0.529619,0.528299,0.547060,0.524997,0.521405,0.522637,0.519659,...,0.500916,0.500971,0.501406,0.501080,0.500591,0.501358,0.500394,0.500420,0.500186,0.500321,0.500070,0.500074,0.502010,0.500138,0.500000,0.500138,0.500016,0.500005,0.500127,0.500476,None,None,0.500058,None,2021-10-05
4,366408690076,0.731059,0.660671,0.681659,0.617803,0.613834,0.595751,0.593028,0.615618,0.583139,0.587001,0.562905,0.570189,0.533386,0.544528,0.533683,0.536379,0.528033,0.529847,0.528471,0.547260,0.525249,0.521641,0.522850,0.520154,...,0.501207,0.501028,0.501694,0.501349,0.500789,0.501696,0.500641,0.500623,0.500434,0.500235,0.500044,0.500350,0.502375,0.500060,0.500128,0.500078,0.500048,0.500001,0.500073,0.500745,None,None,0.500000,None,2021-10-05


In [22]:
query = '''
select max(LOAD_DT) AS MAX_DATE
FROM {}
where inn is not Null
'''.format(ISKRA_TABLE)

In [23]:
max_date_pd = pd.read_sql(query, con=db.connection)

In [24]:
max_date_pd.head()

,MAX_DATE
0,2021-10-05


In [32]:
'2021-09-23'

'2021-09-23'

In [28]:
max_date_load = max_date_pd["MAX_DATE"][0]

In [29]:
max_date_load

'2021-10-05'

In [37]:
max_date_load = '2021-09-23'

In [38]:
query_for_inn_cnt = """
SELECT count(INN) AS INN_CNT
FROM {}
WHERE inn is not Null
  and LOAD_DT = '{}'
""".format(ISKRA_TABLE, max_date_load)

In [43]:
print(query_for_inn_cnt)


SELECT count(distinct INN) AS INN_CNT_DIST
FROM recsys_story_asup_ratings
WHERE inn is not Null
  and LOAD_DT = '2021-09-23'



In [39]:
inn_cnt_dist_pd = pd.read_sql(query_for_inn_cnt, con=db.connection)

In [40]:
inn_cnt_dist = inn_cnt_dist_pd["INN_CNT"][0]

In [41]:
inn_cnt_dist

0

In [48]:
max_date_load

'2021-09-23'

In [16]:
np.unique(df['LOAD_DT'])

array(['2021-09-23'], dtype=object)

# mail sendler 

In [50]:
from lib.mail_sender import *
import json

In [51]:
EMAIL_SETTINGS_FILE = "mail_settings/mail_settings.txt"

In [52]:
with open(EMAIL_SETTINGS_FILE, "r") as f:
    settings = json.load(f)

In [59]:
EMAIL_LIST_FILE = "mail_settings/mail_list.txt"

In [77]:
with open(EMAIL_LIST_FILE, "r") as f:
    recivers_str = f.read()
    receivers = recivers_str.split('\n')

In [78]:
receivers

['EIShubochkin@omega.sbrf.ru', 'EIShubochkin@sberbank.ru']

In [80]:
msg = """
ISKRA:
Последняя дата загрузки: {}
Число уникальных ИНН на последнюю дату: {}
""".format(max_date_load, inn_cnt_dist)

In [81]:
msg

'\nISKRA:\nДата загрузки: 2021-09-23\nЧисло уникальных ИНН: 1184714\n'

In [67]:
authorization = Authorization(user=settings["user"], domain='ALPHA', mailbox=settings["mailbox"],
                              server='cab-vop-mbx2053.omega.sbrf.ru')

In [68]:
mail_receiver = MailReceiver(settings["password"], auth_class=authorization)

In [74]:
date = datetime.strftime(datetime.today(), format='%Y-%m-%d')

In [79]:
mail_receiver.send_message(receivers, 'SBBOL_STORY_RATINGS_STATS {}'.format(date), msg)

# Cheking dataset 

In [12]:
dataset = load_table(SBX_TEAM_DIGITCAMP, ALS_HISTORY_ASUP_RATINGS, hive).cache()

In [14]:
# drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_POSITIVE, hive)

In [ ]:
# drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_POSITIVE, hive)

In [15]:
%%time
# hive.sql("drop table if exists {schema}.{tbl} purge".format(schema=SBX_TEAM_DIGITCAMP, tbl=RECSYS_STORY_DATASET))

CPU times: user 2.65 ms, sys: 1.61 ms, total: 4.26 ms
Wall time: 1.74 s


DataFrame[]

In [13]:
%%time
dataset.show()

CPU times: user 25.6 ms, sys: 78 µs, total: 25.7 ms
Wall time: 3.21 s


,inn,korpkarta,e_inv,rabota_ru,merchant_acquiring,e_acquiring,kredit,credit_card,bip,zarplata,LYurist,sberrating,sppu,customWarranty,mybuch_online_sso,sms,time2pay,samoinkass,deposit,cashOrder,leasing,loyalty_business,evotor,invoice,bki,...,markets,property_management,distressed_assets,structuralDepositRangeAccrual,ofd_platforma,sber_business_bot,CooperateOnline,e_inv_archive,bestbusinessp,packages,taxFree,currency_control_notifications,InSales,dcdGm,customs_duties,gos_oboron_zakaz,derivatives,credit_insurance,brokerage,sber_target,metrics,depositdualcurrency,contcheck,cash_insurance,load_dt
0,010110425588,0.731059,0.660678,0.681667,0.617815,0.613847,0.595803,0.593068,0.615617,0.583162,0.587013,0.562950,0.570226,0.533398,0.544551,0.533717,0.536407,0.528070,0.529885,0.528495,0.547286,0.525282,0.521668,0.522874,0.520214,...,0.501244,0.501038,0.501730,0.501385,0.500813,0.501738,0.500673,0.500650,0.500465,0.500225,0.500046,0.500392,0.502419,0.500055,0.500148,0.500076,0.500059,0.500000,0.500069,0.500776,None,None,0.500014,None,2021-10-05
1,0104015530,0.731059,0.660673,0.681675,0.617867,0.613915,0.595944,0.593180,0.615644,0.583223,0.587050,0.563073,0.570309,0.533423,0.544654,0.533835,0.536473,0.528212,0.529943,0.528557,0.547355,0.525370,0.521787,0.522984,0.520389,...,0.501356,0.501066,0.501848,0.501483,0.500922,0.501893,0.500784,0.500749,0.500571,0.500209,0.500042,0.500463,0.502576,0.500033,0.500189,0.500041,0.500057,0.500014,0.500054,0.500906,None,None,0.500000,None,2021-10-05
2,0105041653,0.731059,0.660685,0.681615,0.617700,0.613695,0.595169,0.592579,0.615620,0.582959,0.586898,0.562457,0.569946,0.533303,0.544273,0.533339,0.536134,0.527660,0.529590,0.528275,0.547017,0.524941,0.521345,0.522583,0.519514,...,0.500849,0.500990,0.501336,0.501026,0.500550,0.501267,0.500339,0.500382,0.500131,0.500387,0.500125,0.500029,0.501911,0.500211,0.500000,0.500206,0.500056,0.500036,0.500188,0.500407,None,None,0.500136,None,2021-10-05
3,010512937730,0.731059,0.660686,0.681613,0.617699,0.613694,0.595143,0.592561,0.615628,0.582955,0.586898,0.562438,0.569938,0.533307,0.544270,0.533329,0.536127,0.527649,0.529579,0.528271,0.547011,0.524932,0.521340,0.522579,0.519488,...,0.500839,0.500997,0.501327,0.501017,0.500548,0.501255,0.500333,0.500379,0.500124,0.500405,0.500136,0.500017,0.501897,0.500226,0.500000,0.500219,0.500063,0.500049,0.500202,0.500400,None,None,0.500150,None,2021-10-05
4,010516506407,0.731059,0.660868,0.681648,0.617799,0.613745,0.594488,0.592138,0.615916,0.583006,0.587072,0.562083,0.569910,0.533637,0.544298,0.533226,0.536175,0.527513,0.529609,0.528419,0.547042,0.524905,0.521352,0.522636,0.518913,...,0.500779,0.501481,0.501270,0.501013,0.500702,0.501083,0.500376,0.500523,0.500165,0.501219,0.500813,0.500000,0.501665,0.501022,0.500322,0.500973,0.500611,0.500665,0.500942,0.500389,None,None,0.500933,None,2021-10-05
5,010600037520,0.731059,0.660699,0.681720,0.617919,0.613973,0.596263,0.593436,0.615648,0.583340,0.587140,0.563324,0.570444,0.533541,0.544782,0.534019,0.536659,0.528384,0.530153,0.528713,0.547529,0.525576,0.521934,0.523121,0.520734,...,0.501575,0.501155,0.502058,0.501696,0.501046,0.502101,0.500958,0.500892,0.500756,0.500213,0.500086,0.500721,0.502810,0.500049,0.500334,0.500098,0.500158,0.500066,0.500089,0.501080,None,None,0.500000,None,2021-10-05
6,010600083277,0.731059,0.662114,0.681941,0.618460,0.614051,0.590537,0.589634,0.617756,0.583431,0.588307,0.559948,0.569746,0.535937,0.544494,0.532629,0.536677,0.526636,0.529997,0.529545,0.547426,0.524916,0.521473,0.523019,0.515414,...,0.500510,0.504807,0.501021,0.501134,0.501714,0.500011,0.500751,0.501531,0.500553,0.506710,0.505330,0.500084,0.500248,0.506324,0.502559,0.506107,0.504349,0.504793,0.505939,0.500377,None,None,0.506159,None,2021-10-05
7,0107008718,0.731059,0.660710,0.681618,0.617720,0.613707,0.595056,0.592507,0.615679,0.582964,0.586933,0.562387,0.569919,0.533362,0.544284,0.533319,0.536138,0.527632,0.529573,0.528299,0.547024,0.524934,0.521353,0.522599,0.519407,...,0.500833,0.501085,0.50

In [15]:
%%time
dataset.select("inn").distinct().count()

CPU times: user 133 ms, sys: 156 ms, total: 289 ms
Wall time: 13min 27s


1189687

In [42]:
dataset.filter("load_dt = '2021-10-05'").select("inn").distinct().count()

1189687

In [43]:
dataset.filter("load_dt = '2021-09-23'").select("inn").distinct().count()

0

# Checking create date 

In [49]:
hive.sql("describe formatted {}.{}".format(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_TARGET)).show()

,col_name,data_type,comment
0,user_id,int,None
1,item_id,int,None
2,timestamp,bigint,None
3,rating,double,None
4,,,
5,# Detailed Table Information,,
6,Database,sbx_team_digitcamp,
7,Table,recsys_story_dataset_target,
8,Owner,shubochkin1-ei_ca-sbrf-ru@DF.SBRF.RU,
9,Created Time,Tue Oct 05 15:24:06 MSK 2021,


In [ ]:
from py4j.protocol import Py4JJavaError

In [3]:
table_info = hive.sql("show create table {}.{}".format(SBX_TEAM_DIGITCAMP, ALS_ASUP_PIVOT)).toPandas().createtab_stmt[0]

In [10]:
target_dataset = load_table(SBX_TEAM_DIGITCAMP, ALS_ASUP_PIVOT, hive).cache()

In [8]:
%%time
target_dataset.select("inn").distinct().count()

CPU times: user 2.38 ms, sys: 5.42 ms, total: 7.8 ms
Wall time: 8.03 s


1189687

In [11]:
%%time
target_dataset.select("inn").distinct().count()

CPU times: user 93.2 ms, sys: 120 ms, total: 213 ms
Wall time: 10min 6s


1189687

In [9]:
target_dataset.select("item_id").distinct().count()

87

In [6]:
target_dataset.show()

,user_id,item_id,timestamp,rating
0,1,382389508,None,0.0
1,8,382545223,None,0.0
2,12,382364619,None,0.0
3,14,382391476,None,0.0
4,29,382393978,None,0.0
5,41,382348021,None,0.0
6,58,382364633,None,0.0
7,62,382393978,None,0.0
8,66,382545223,None,0.0
9,67,382567886,None,0.0


In [17]:
from datetime import datetime, timedelta

In [6]:
import re

In [5]:
table_info

"CREATE TABLE `sbx_team_digitcamp`.`RECSYS_STORY_DATASET_TARGET`(`user_id` INT, `item_id` INT, `timestamp` BIGINT, `rating` DOUBLE)\nROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'\nWITH SERDEPROPERTIES (\n  'serialization.format' = '1'\n)\nSTORED AS\n  INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat'\n  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'\nTBLPROPERTIES (\n  'transient_lastDdlTime' = '1633447886'\n)\n"

In [7]:
transient_lastDdlTime = re.search(r"'transient_lastDdlTime' = '[0-9]*'" , table_info).group()

In [8]:
create_time_str = re.search(r"[0-9]{4,20}", transient_lastDdlTime).group()

In [9]:
create_time_int = int(create_time_str)

In [13]:
import time

In [14]:
from datetime import datetime

In [10]:
create_dt = datetime.fromtimestamp(create_time_int)

In [11]:
create_dt

datetime.datetime(2021, 10, 5, 18, 31, 26)

In [18]:
curr_dt = datetime.today()

In [28]:
curr_dt

datetime.datetime(2021, 10, 5, 16, 43, 45, 759400)

In [62]:
create_dt

datetime.datetime(2021, 10, 5, 17, 51, 5)

In [21]:
type(curr_dt)

datetime.datetime

In [36]:
datetime.date(4, 10, 2021)

TypeError: descriptor 'date' requires a 'datetime.datetime' object but received a 'int'

In [35]:
curr_dt - datetime.cr

TypeError: descriptor 'date' requires a 'datetime.datetime' object but received a 'int'

In [41]:
delta = (curr_dt - create_dt).days

In [44]:
delta <= 2

True

In [33]:
delta.days

0